In [72]:
"""
DSCI-663-03 Project: Data Preprocessing File
This file preforms some of our discrete preprocessing tasks for our ML

:language:      Python with pandas
:author:        Stephen Cook
:author:        Cory Maclauchlan
:author:        Robert Gentilucci
:author:        Julia Okvath
:date created:  10/18/21
:last edit:     11/27/21
"""
# import pandas and numpy
import pandas as pd
import category_encoders as ce

In [73]:
# Task 1: Load in the data file, mental_health_data.csv
mental_health_data_filename = 'mental_health_data.csv'
mental_health_df= pd.read_csv(mental_health_data_filename)

In [74]:
# Task 2: Drop all records for self employed people
# This was done because self-employed respondents are irrelevant to our questions
mental_health_df.drop(mental_health_df[mental_health_df['A'] == 1].index, inplace = True)


In [75]:
# Task 3: Preform Feature Subset Selection by filtering out irrelevant columns
mental_health_df = mental_health_df.drop(columns = ['T','Q', 'R', 'S', 'U', 'V', 'W', 'X', 'A', 'D', 'L', 'O', 'LL', 'NN', 
                                                    'WW', 'XX', 'ZZ', 'BBB', 'CCC', 'FFF', 'JJJ', 'GGG', 'III', 'KK','FF',
                                                    'C', 'Y','Z', 'AA','BB','CC','EE','GG','HH','II','JJ'])


In [76]:
## Task 4: Replace General Missing Values

no_Entry = {'SS': 'No Entry', 'GGG': 'No Entry', 'III': 'No Entry', 'EEE': 'No Entry', 'RR': 'No Entry', 'F': 'No Entry'}
mental_health_df = mental_health_df.fillna(value = no_Entry)

not_App = {'Z': 'Not Applicable', 'AA': 'Not Applicable', 'BB': 'Not Applicable','CC': 'Not Applicable',
           'DD': 'Not Applicable','EE': 'Not Applicable', 'FF': 'Not Applicable','GG': 'Not Applicable',
           'HH': 'Not Applicable', 'II': 'Not Applicable', 'JJ': 'Not Applicable', 'Y': 'Not Applicable'}
mental_health_df = mental_health_df.fillna(value = not_App)

In [77]:
# Task 5: Standardize the Gender Responses, as they were free response.

# Var for the gender column ID
gender_col_id = "EEE"

# for this method to work better, strip leading and ending whitespace
mental_health_df[gender_col_id] = mental_health_df[gender_col_id].str.strip()

# regex explanation:
# (?i) - case insensitive, looks for female or woman or f
female_regex_pattern = r'(?i).*(female|woman|f).*'

# regex explanation:
# (?i) - case insensitive, looks for male or man or m
male_regex_pattern = r'(?i).*(male|man|m).*'

# regex explanation:
# any value, not F or M gets O
other_regex_pattern = r'^(?!M).*^(?!F).*'

# replace female
mental_health_df[gender_col_id].replace(to_replace=female_regex_pattern, value = 'F',
                                        inplace = True, regex= True)
# replace male
mental_health_df[gender_col_id].replace(to_replace=male_regex_pattern, value = 'M',
                                        inplace = True, regex= True)

# replace other
mental_health_df[gender_col_id].replace(to_replace=other_regex_pattern, value = 'O',
                                        inplace = True, regex= True)

#fill na spaces with other.
mental_health_df[gender_col_id] = mental_health_df[gender_col_id].fillna(value="O")# This var makes my life a little easier

In [78]:
# Task 6: Manage Age Outliers

# Find IQR of age group for removal of extreme outliers
Q1 = mental_health_df['DDD'].quantile(.25)
Q3 = mental_health_df['DDD'].quantile(.75)
IQR = Q3-Q1

# Remove extreme outliers in the age column
# Note IQR is used for upper bound; however, IQR is invalid for lowered as it would be negative. we use 15
mental_health_df.drop(mental_health_df[(mental_health_df['DDD'] > Q3+6*IQR)
                                       | (mental_health_df['DDD'] < 15)].index, axis = 0, inplace=True)

In [79]:
# Task 7: Discretize age data

mental_health_df['DDD'] = pd.cut(mental_health_df["DDD"], bins=[0, 25, 35, 45, 100],
                                 labels=["<25", "25-35", "36-45", ">45"])



In [80]:
# Task 8: Cast all binary columns to boolean type, this caused issues in Apriori
mental_health_df['AAA'] = mental_health_df['AAA'].astype('bool')


In [81]:
# Task 9: Encode chosen columns to binary values for later aggregation

encoder_bin = [
    ce.OrdinalEncoder(cols=['E'],return_df=True,mapping=[{'col':'E','mapping':{'No':0,'Yes':1,"I don't know":0,'Not eligible for coverage / N/A':0}}]),
    ce.OrdinalEncoder(cols=['F'],return_df=True,mapping=[{'col':'F','mapping':{'No':0,'Yes':1,"No Entry":0,'I am not sure':0}}]),
    ce.OrdinalEncoder(cols=['G'],return_df=True,mapping=[{'col':'G','mapping':{'No':0,'Yes':1,"I don't know":0}}]),
    ce.OrdinalEncoder(cols=['H'],return_df=True,mapping=[{'col':'H','mapping':{'No':0,'Yes':1,"I don't know":0}}]),
    ce.OrdinalEncoder(cols=['I'],return_df=True,mapping=[{'col':'I','mapping':{'No':0,'Yes':1,"I don't know":0}}]),
    ce.OrdinalEncoder(cols=['J'],return_df=True,mapping=[{'col':'J','mapping':{'Very easy':1,'Somewhat easy':1,"I don't know":0,
                                                          'Neither easy nor difficult':1, 'Very difficult':0,
                                                          'Somewhat difficult':0}}]),
    ce.OrdinalEncoder(cols=['K'],return_df=True,mapping=[{'col':'K','mapping':{'No':1,'Yes':0,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['M'],return_df=True,mapping=[{'col':'M','mapping':{'No':0,'Yes':1,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['N'],return_df=True,mapping=[{'col':'N','mapping':{'No':0,'Yes':1,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['P'],return_df=True,mapping=[{'col':'P','mapping':{'No':1,'Yes':0}}]),
    ce.OrdinalEncoder(cols=['DD'],return_df=True, mapping=[{'col':'DD','mapping':{'No':0,'Yes, always':1,"I don't know":0,'Sometimes':0,
                                                          'Not Applicable':0}}]),
    ce.OrdinalEncoder(cols=['MM'],return_df=True, mapping=[{'col':'MM','mapping':{'No':0,'Yes':1,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['OO'],return_df=True,
                           mapping=[{'col':'OO','mapping':{"No, I don't think it would":1,"No, it has not":1,
                                                           'Yes, I think it would':0,'Yes, it has':0,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['PP'],return_df=True,
                           mapping=[{'col':'PP','mapping':{"No, I don't think they would":1,"No, they do not":1,
                                                           'Yes, I think they would':0,'Yes, they do':0,"Maybe":0}}]),
    ce.OrdinalEncoder(cols=['QQ'],return_df=True,
                           mapping=[{'col':'QQ','mapping':{"Somewhat open":1,
                                                           "Not applicable to me (I do not have a mental illness)":0,
                                                           'Very open':1,'Not open at all':0,"Neutral":0,
                                                           'Somewhat not open':0}}]),
    ce.OrdinalEncoder(cols=['RR'],return_df=True,
                           mapping=[{'col':'RR','mapping':{"No":1,"Maybe/Not sure":0,'Yes, I experienced':0,
                                                           'Yes, I observed':0,"No Entry":0}}]),
    ce.OrdinalEncoder(cols=['SS'],return_df=True,
                           mapping=[{'col':'SS','mapping':{"No":1,"Maybe":0,'Yes':0,"No Entry":0}}])





]
for encoder in encoder_bin:
    mental_health_df = encoder.fit_transform(mental_health_df)


In [82]:
# Task 10: Sum relevant columns to create indicators for "Workplace Culture", "Employee comfort with mental health in the
# workplace", and "Employee comfort with mental health"
# The purpose of this is to reduce dimensionality of the survey data by encoding survey questions for each of the 3 
# indicators listed and summing them
mental_health_df['Workplace_Culture'] = (mental_health_df['E'] +
                                                       mental_health_df['F'] +
                                                       mental_health_df['G'] +
                                                       mental_health_df['H'] +
                                                       mental_health_df['I'] +
                                                       mental_health_df['J'] +
                                                       mental_health_df['P'] +
                                                       mental_health_df['DD'] +
                                                       mental_health_df['RR'] +
                                                       mental_health_df['SS'])


mental_health_df['Employee_Comfort_w/_MH_@_Workplace'] = (mental_health_df['PP'] +
                                                                        mental_health_df['OO'] +
                                                                        mental_health_df['M'] +
                                                                        mental_health_df['N'] +
                                                                        mental_health_df['K'])

mental_health_df['Employee_Comfort_w/_MH'] = (mental_health_df['MM'] +
                                                            mental_health_df['QQ'])

In [83]:
# Task 11: drop columns which were used to create the 3 indicator columns:
# "Workplace Culture", "Employee comfort with mental health in the workplace", and "Employee comfort with mental health"

mental_health_df = mental_health_df.drop(columns = ['E', 'F', 'G', 'H', 'I', 'J', 'K',
                                                                 'M', 'N', 'P', 'DD', 'MM', 'OO', 'PP',
                                                                 'QQ', 'RR', 'SS'])

In [84]:
# Review summary statistics for use in column discretization (for association rule mining)
# The bins will be based on the mean of each indicator. The bin labels will be "Below Average", "Average", or "Above Average"
# The bin width for the "Average" label will be the mean of the data +/- 1 sigma. If the Average - 1 sigma results in a
# negative number then only 2 bins will be used to discretize the data -> "Average" and "Above Average"
mental_health_df.describe()

,Workplace_Culture,Employee_Comfort_w/_MH_@_Workplace,Employee_Comfort_w/_MH
count,1143.000000,1143.000000,1143.000000
mean,3.694663,1.422572,0.701662
std,1.890255,1.535025,0.568640
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,3.000000,1.000000,1.000000
75%,5.000000,3.000000,1.000000
max,9.000000,5.000000,2.000000


In [85]:
# Task 12: discretize the "Workplace Culture", "Employee comfort with mental health in the workplace", 
# and "Employee comfort with mental health" columns so data can be used with the Apriori algorithm

binsWC=[-1, 1.8, 5.58, 9]
labelsWC=['Below_Avg', 'Average', 'Above Average']

mental_health_df_redux['Workplace_Culture_discrete'] = pd.cut(mental_health_df_redux['Workplace_Culture'], 
                                                               binsWC, labels=labelsWC)

binsMHW=[-1, 3, 5]
labelsMHW=['Average', 'Above Average']

mental_health_df_redux['Employee_Comfort_w/_MH_@_Workplace_discrete'] = pd.cut(mental_health_df_redux['Employee_Comfort_w/_MH_@_Workplace'], 
                                                                                binsMHW, labels=labelsMHW)

binsMH=[-1, 1, 2]
labelsMH=['Average', 'Above Average']

mental_health_df_redux['Employee_Comfort_w/_MH_discrete'] = pd.cut(mental_health_df_redux['Employee_Comfort_w/_MH'], 
                                                                                binsMH, labels=labelsMH)

In [86]:
# Task 13: print the cleaned file to csv
mental_health_df_redux.to_csv('mental_health_CLEAN.csv', index=False, quoting=1)

